## **INSTRUÇÕES**

Codigo para juntar os bids de BR e US.
---
### Todos os arquivos baixados devem ser upados na pasta 'Arquivos de entrada'
---

As tabelas de entrada para esse script devem ter a seguinte estrutura:
* **coluna "sku"**: contém os SKUs dos hotéis e deve ter nome: "sku". 
* **coluna do bid**: contém o valor do bid e **deve ser nomeada como "BR" ou "US"**, dependendo do POS desejado

#### **Passo 1:**  Subir os bids de BR para a pasta de arquivo de entrada. Essa tabela deve ter apenas duas colunas:


Elabore a tabela da forma explicitada e nomeie-a como:

* BID BR Futuro.csv

#### **Passo 2:**  Subir os bids de US para a pasta de arquivo de entrada

* BID US Futuro.csv


# IMPORTANTE


Quando for rodar o script, você precisa clicar no link azul da célula e autoriza o colab à acessar seu Drive, pra fazer isso, copie o código que vai aparecer quando você clicar no link e cole ele na caixa de texto que vai aparecer quando você rodar a célula abaixo

In [1]:
 from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


#### Importando Dados


In [2]:
### importando bibliotecas que iremos utilizar
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
### Dados
dadosBR = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/BID BR Futuro.csv', delimiter = ';',encoding='windows-1254')).fillna(0)
dadosUS = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/BID US Futuro.csv', delimiter = ';',encoding='windows-1254')).fillna(0)

bid_hoje = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/bid live hoje.csv', delimiter = ',')).fillna(0)


In [4]:
dadosUS.head()
#dadosUS.head()

,ï»¿sku,US
0,JNP-JP06450R,17
1,JNP-JP055209,15
2,JNP-JP312267,40
3,JNP-JP981787,40
4,JNP-JP735105,15


In [5]:
bid_hoje.head()

,partner_reference,BR,US
0,JNP-JP771852,37,42
1,JNP-JP244388,15,18
2,JNP-JP998891,0,18
3,JNP-JP053448,4,18
4,JNP-JP192959,12,13


In [6]:
dadosBR = dadosBR.rename(columns = {'ï»¿sku':'sku'})
dadosUS = dadosUS.rename(columns = {'ï»¿sku':'sku'})

display(dadosUS)

,sku,US
0,JNP-JP06450R,17
1,JNP-JP055209,15
2,JNP-JP312267,40
3,JNP-JP981787,40
4,JNP-JP735105,15
...,...,...
108688,JNP-JP074552,15
108689,JNP-JP074544,49
108690,JNP-JP074521,17
108691,JNP-JP074515,49


In [7]:
#Juntando os BIDS

dadosjuntos = pd.merge(dadosBR, dadosUS, how = 'outer', on = 'sku') # outer join para manter todos os bids de todos os lados
dadosjuntos = dadosjuntos.fillna(0)
dadosjuntos.loc[:, 'US'] =  dadosjuntos.loc[:, 'US'].astype('int') # changing the type of column from float to int because of bug in pandas.
dadosjuntos

,sku,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49
...,...,...,...
108692,JNP-JP077769,67,48
108693,JNP-JP077766,67,49
108694,JNP-JP077761,0,0
108695,JNP-JP077757,67,49


In [8]:
dadosjuntos.head()

,sku,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49


In [9]:
# Renomear coluna

dadosjuntos = dadosjuntos.rename(columns={'sku': 'partner_reference'})   # a coluna "partner_reference" agora chamara "sku"
bid_hoje = bid_hoje.rename(columns={'BR': 'BR_Hoje'})   # a coluna "partner_reference" agora chamara "sku"
bid_hoje = bid_hoje.rename(columns={'US': 'US_Hoje'})   # a coluna "partner_reference" agora chamara "sku"


In [10]:
dadosjuntos.head()

,partner_reference,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49


In [11]:
#Juntando os BIDS + bid hoje para fazer o delta

Feed = pd.merge(dadosjuntos, bid_hoje, how = 'left', on = 'partner_reference') #aplicando left join. ou seja mantera todos os dados da esquerda.


In [12]:
Feed.head()

,partner_reference,BR,US,BR_Hoje,US_Hoje
0,OMN-4273,3,14,7,11
1,OMN-3185,15,46,1,39
2,JNP-JP408570,16,20,19,13
3,OMN-4480,5,48,14,41
4,JNP-JP335495,54,49,58,42


In [13]:
# Criando coluna delta 

Feed['delta_BR'] = Feed['BR'] - Feed['BR_Hoje']
Feed['delta_US'] = Feed['US'] - Feed['US_Hoje'] 

In [14]:
print('MENORES DELTAS BR')
display(Feed.sort_values(by = ['delta_BR']).head())
print()
print('MAIORES DELTAS BR')
display(Feed.sort_values(by = ['delta_BR']).tail())
print()

print('MENORES DELTAS US')
display(Feed.sort_values(by = ['delta_US']).head())
print()
print('MAIORES DELTAS US')
display(Feed.sort_values(by = ['delta_US']).tail())


MENORES DELTAS BR


,partner_reference,BR,US,BR_Hoje,US_Hoje,delta_BR,delta_US
37,JNP-JP052941,1,25,11,18,-10,7
68,JNP-JP843547,5,25,15,18,-10,7
59,JNP-JP02630H,13,25,23,18,-10,7
39,OMN-2756,34,34,44,31,-10,3
73,JNP-JP053559,50,46,60,39,-10,7



MAIORES DELTAS BR


,partner_reference,BR,US,BR_Hoje,US_Hoje,delta_BR,delta_US
72972,OMN-12871,15,49,0,42,15,7
6874,JNP-JP354934,15,2,0,3,15,-1
74669,OMN-5969,15,49,0,42,15,7
70979,JNP-JP006510,15,49,0,42,15,7
74640,OMN-6192,15,46,0,39,15,7



MENORES DELTAS US


,partner_reference,BR,US,BR_Hoje,US_Hoje,delta_BR,delta_US
63836,JNP-JP055209,67,15,60,26,7,-11
45929,JNP-JP657275,68,29,61,38,7,-9
57845,JNP-JP040264,68,54,61,63,7,-9
107697,JNP-JP081188,68,7,61,14,7,-7
24627,JNP-JP277487,68,6,61,13,7,-7



MAIORES DELTAS US


,partner_reference,BR,US,BR_Hoje,US_Hoje,delta_BR,delta_US
69571,JNP-JP002281,68,15,61,0,7,15
51874,JNP-JP781936,68,15,61,0,7,15
29055,JNP-JP870265,68,15,61,0,7,15
15252,JNP-JP234589,67,15,60,0,7,15
15285,JNP-JP234075,68,15,61,0,7,15


In [15]:
dadosjuntos.loc[:, ['BR', 'US']] = dadosjuntos[['BR', 'US']].astype(int)
dadosjuntos.head()

,partner_reference,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49


In [16]:
dadosjuntos

,partner_reference,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49
...,...,...,...
108692,JNP-JP077769,67,48
108693,JNP-JP077766,67,49
108694,JNP-JP077761,0,0
108695,JNP-JP077757,67,49


In [17]:
# CUSPINDO PLANILHA FINAL
dia = (dt.datetime.today() + dt.timedelta(days=1)).strftime(format='%d_%m_%y')

dadosjuntos.drop_duplicates(inplace = True)
dadosjuntos.drop_duplicates().to_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/bid_' + dia + '.csv', sep=',', decimal='.', 
                              encoding="utf-8-sig", index = False, columns = ['partner_reference', 'BR', 'US'])

In [18]:
dadosjuntos

,partner_reference,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49
...,...,...,...
108692,JNP-JP077769,67,48
108693,JNP-JP077766,67,49
108694,JNP-JP077761,0,0
108695,JNP-JP077757,67,49
